In [ ]:
!pip install mne
import mne
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import kurtosis
import scipy
from scipy import stats
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import scipy as sp
!pip install mne networkx
import networkx as nx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_data(file_path):
  raw = mne.io.read_raw_cnt(file_path, preload=True)
  # content = content.drop_channels(["T","X","Y"])
  f_channels = [ch_name for ch_name in raw.ch_names if ch_name.startswith('F')]
  # Extract the data from the "F" channels
  raw_f = raw.copy().pick_channels(f_channels)
  eeg_data = raw_f.pick_types(eeg=True).get_data()
  df = pd.DataFrame(eeg_data.T, columns=raw_f.ch_names)
  print(df)
  # Preprocess
  raw.filter(l_freq=1, h_freq=40)
  # For example, to remove eye blinks using ICA
  ica = mne.preprocessing.ICA(n_components=20, random_state=42)
  ica.fit(raw)
  ica.exclude = [0, 1]  # indices of ICs to exclude
  raw = ica.apply(raw)


  raw.set_eeg_reference(ref_channels='average')
  raw.set_eeg_reference(ref_channels='average', projection=True)

  annotations = mne.Annotations(onset=[1, 5, 9], duration=[1, 1, 1], description=['Event 1', 'Event 2', 'Event 3'])
  raw.set_annotations(annotations)

  # Extract events from the annotations
  events, event_id = mne.events_from_annotations(raw)
  # raw.annotations
  # print(event_id)

  # stim_channel = 'HEOL'
  # events = mne.find_events(raw,stim_channel=stim_channel)
  # event_id = {'event_name': 1}
  tmin, tmax = -0.2, 0.5
  try:
      # events = mne.find_events(raw)
      epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=0.5, baseline=None)
  except ValueError as e:
      print("Error:", e)
  print(events)
  return raw


In [ ]:
from glob import glob
all_file_path=glob("/content/drive/MyDrive/cntfiles/Raw Data/*/*/*.cnt")
# all_file_path=glob("/content/drive/MyDrive/setfiles/*.set")
print(all_file_path)

['/content/drive/MyDrive/cntfiles/Raw Data/6/6/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/6/6/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/12/12/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/12/12/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/11/11/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/11/11/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/4/4/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/4/4/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/3/3/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/3/3/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/5/5/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/5/5/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/2/2/Normal state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/2/2/Fatigue state.cnt', '/content/drive/MyDrive/cntfiles/Raw Data/7/7/Fatigue state.cnt', '/conten

In [ ]:
sample_data=[read_data(i) for i in all_file_path]
#data=read_data("/content/drive/MyDrive/preprocessed/sub0.set")
# content=mne.io.read_raw_eeglab("/content/drive/MyDrive/preprocessed/sub0.set", eog=(), preload=True)
# data=content.get_data()
# print(len(sample_data))


NameError: ignored

In [ ]:
raw = mne.io.read_raw_cnt('/content/drive/MyDrive/cntfiles/Raw Data/2/2/Fatigue state.cnt', preload=True)

In [ ]:
print(raw.info)

In [ ]:
scalings= {'eeg': 1e-3}
raw.plot(n_channels=40, scalings=scalings, title='EEG Data', block=True)

In [ ]:
data, times = raw[:]

# Create figure and axis objects
fig, ax = plt.subplots(figsize=(20,10))

# Plot EEG data with adjusted scalings
ax.plot(times, data.T * 100e-2)  # Adjust the scaling factor (100e-6) to your preference

# Set axis labels and title
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude (V)')
ax.set_title('EEG Data')

# Show plot
plt.show()


In [ ]:
f_channels = [ch_name for ch_name in raw.ch_names if ch_name.startswith('F')]

# Extract the data from the "F" channels
raw_f = raw.copy().pick_channels(f_channels)
eeg_data = raw_f.pick_types(eeg=True).get_data()
df = pd.DataFrame(eeg_data.T, columns=raw_f.ch_names)
df
# Plot the data from the "F" channels
# raw_f.plot(scalings=scalings, title='EEG Data')


In [ ]:
# Preprocess
raw.filter(l_freq=1, h_freq=40)
# For example, to remove eye blinks using ICA
ica = mne.preprocessing.ICA(n_components=20, random_state=42)
ica.fit(raw)
ica.exclude = [0, 1]  # indices of ICs to exclude
raw = ica.apply(raw)


raw.set_eeg_reference(ref_channels='average')
raw.set_eeg_reference(ref_channels='average', projection=True)

annotations = mne.Annotations(onset=[1, 5, 9], duration=[1, 1, 1], description=['Event 1', 'Event 2', 'Event 3'])
raw.set_annotations(annotations)

# Extract events from the annotations
events, event_id = mne.events_from_annotations(raw)
# raw.annotations
# print(event_id)

# stim_channel = 'HEOL'
# events = mne.find_events(raw,stim_channel=stim_channel)
# event_id = {'event_name': 1}
tmin, tmax = -0.2, 0.5
try:
    # events = mne.find_events(raw)
    epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=0.5, baseline=None)
except ValueError as e:
    print("Error:", e)
print(events)

In [ ]:
mne.viz.plot_raw(raw)

In [ ]:
stim_channel = 'FP1'
events = mne.find_events(raw,stim_channel=stim_channel)
print(events)

In [ ]:
print(sample_data[0])

In [ ]:
df = pd.DataFrame(sample_data[0].T, columns=raw_f.ch_names)
df

In [ ]:
mne.viz.plot_raw(sample_data[0])